In [65]:
import pandas as pd
import numpy as np
from sklearn.manifold import TSNE

from utils import *

In [66]:
# Load data 
path = '.\data\stock_port.csv'
df = load_data(path)

# Convert df to sparse matrix
sp_matrix, row_ind_dict, col_ind_dict = convert_data_sparse_matrix(df)

# Basic Info
print('Dimension of sparse_matrix is ', sp_matrix.shape)
row_dim = sp_matrix.shape[0]
col_dim = sp_matrix.shape[1]

# Calculate shareholding % by stock_code
sp_matrix_stock = sp_matrix / np.sum(sp_matrix, axis = 1).reshape(row_dim, -1)

# Calculate shareholding % by shareholder
# sp_matrix_shareholder = sp_matrix / np.sum(sp_matrix, axis = 1).reshape(row_dim, -1)
sp_matrix_shareholder = sp_matrix / np.sum(sp_matrix, axis = 0).reshape(-1, col_dim)

# Element-wise multiply two matrix
sp_matrix_stock_shareholder = sp_matrix_stock * sp_matrix_shareholder

Dimension of sparse_matrix is  (2020, 709)


In [127]:
# Apply TSNE to sp_matrix_stock_shareholder
X_embedded = TSNE(n_components = 2, perplexity = 100, learning_rate = 500).fit_transform(sp_matrix_stock_shareholder)



In [128]:
df_tsne = pd.DataFrame(X_embedded, columns = ['X1', 'X2'])
df_tsne['stock_code'] = df_tsne.reset_index()['index'].apply(lambda x: {j:i for i,j in row_ind_dict.items()}[x])

In [129]:
df_tsne
highlight_stock_code = input('Stock Code: ')
df_tsne['highlight'] = df_tsne['stock_code'].apply(lambda x: 1 if x == highlight_stock_code else 0)
df_tsne

Stock Code: 00001


,X1,X2,stock_code,highlight
0,12.123413,-16.841549,00001,1
1,11.767634,-17.025877,00002,0
2,11.767634,-17.025877,00003,0
3,11.999039,-17.445990,00004,0
4,12.385491,-16.968069,00005,0
...,...,...,...,...
2015,-3.179542,0.530712,04609,0
2016,-2.694836,1.622579,04610,0
2017,-2.694816,1.622573,04611,0
2018,-2.308873,1.215470,04612,0


In [130]:
import plotly.express as px

fig = px.scatter(df_tsne,'X1','X2', hover_name = 'stock_code', color = 'highlight')
fig.add_scatter()
fig.show()


In [48]:
# Find out row that has > 50% of its shareholding held by a single shareholder
(sp_matrix_stock > 0.01).sum()

25900

In [46]:
sp_matrix_shareholder.max()

0.1

0.0

In [54]:
sp_matrix_stock.max()

0.915131261586699

In [56]:
ind = np.unravel_index(np.argmax(sp_matrix_stock, axis  = None), sp_matrix_stock.shape)

In [62]:
{j:i for i, j in row_ind_dict.items()}[ind[0]]

'00973'

In [63]:
{j:i for i, j in col_ind_dict.items()}[ind[1]]

'HONG KONG INTERNATIONAL SECURITIES LTD'

In [64]:
sp_matrix_shareholder[ind[0], ind[1]]

0.09141613085495164

In [30]:
df = pd.read_csv('stock_port.csv')
df['stock_code'] = df['stock_code'].apply(lambda x: ('00000' + str(x))[-5:])

In [31]:
df.head()

,name_of_ccass_participant,stock_code,shareholding
0,THE HONGKONG AND SHANGHAI BANKING,00005,2622440839
1,CHINA SECURITIES DEPOSITORY AND CLEARING,00005,1864613717
2,BANK OF CHINA (HONG KONG) LTD,00005,1003610032
3,CITIBANK N.A.,00005,661483817
4,HANG SENG BANK LTD,00005,541534277


In [32]:
df['name_of_ccass_participant'].unique()

array(['THE HONGKONG AND SHANGHAI BANKING',
       'CHINA SECURITIES DEPOSITORY AND CLEARING',
       'BANK OF CHINA (HONG KONG) LTD', 'CITIBANK N.A.',
       'HANG SENG BANK LTD', 'STANDARD CHARTERED BANK (HONG KONG) LTD',
       'HANG SENG SECURITIES LTD', 'UBS SECURITIES HONG KONG LTD',
       'BOCI SECURITIES LTD', 'JPMORGAN CHASE BANK, NATIONAL',
       'HSBC BROKING SECURITIES (HONG KONG) LTD',
       'CMB WING LUNG BANK LTD', 'THE BANK OF EAST ASIA LTD',
       'SHANGHAI COMMERCIAL BANK LTD',
       'BANK OF COMMUNICATIONS TRUSTEE LTD', 'DBS BANK (HONG KONG) LTD',
       'DEUTSCHE BANK AG', 'BNP PARIBAS SECURITIES SERVICES',
       'NANYANG COMMERCIAL BANK LTD', 'OCBC WING HANG BANK LTD',
       'UOB KAY HIAN (HONG KONG) LTD', 'EAST ASIA SECURITIES CO LTD',
       'GOLDMAN SACHS (ASIA) SECURITIES LTD', 'DAH SING SECURITIES LTD',
       'ICBC (ASIA) SECURITIES LTD', 'CHONG HING SECURITIES LTD',
       'CHINA CITIC BANK INTERNATIONAL LTD',
       'CHIYU BANKING CORPORATION LTD', '

In [33]:
# Prepare zeros matrix
row_dim = len(df['stock_code'].unique())
col_dim = len(df['name_of_ccass_participant'].unique())

print('Row dimension: ', row_dim)
print('Column dimension: ', col_dim)

sparse_matrix = np.zeros((row_dim, col_dim))


Row dimension:  2020
Column dimension:  709


In [37]:
# Prepare label to index dictionaries
# One for stock_code, one for name of ccass_participant

stock_code_ind = {stock_code:ind for ind, stock_code in enumerate(sorted(df['stock_code'].unique().tolist()))}
shareholder_ind = {shareholder:ind for ind, shareholder in enumerate(sorted(df['name_of_ccass_participant'].unique().tolist()))}

In [38]:
# apply the dict to df
df['stock_code_ind'] = df['stock_code'].apply(lambda x: stock_code_ind[x])
df['shareholder_ind'] = df['name_of_ccass_participant'].apply(lambda x: shareholder_ind[x])

In [39]:
df

,name_of_ccass_participant,stock_code,shareholding,stock_code_ind,shareholder_ind
0,THE HONGKONG AND SHANGHAI BANKING,00005,2622440839,4,600
1,CHINA SECURITIES DEPOSITORY AND CLEARING,00005,1864613717,4,106
2,BANK OF CHINA (HONG KONG) LTD,00005,1003610032,4,27
3,CITIBANK N.A.,00005,661483817,4,133
4,HANG SENG BANK LTD,00005,541534277,4,273
...,...,...,...,...,...
375745,CITIC SECURITIES BROKERAGE (HK) LTD,04612,100,2018,134
375746,BANK OF COMMUNICATIONS TRUSTEE LTD,04612,40,2018,28
375747,CMB WING LUNG BANK LTD,04612,40,2018,138
375748,BOCI SECURITIES LTD,04612,30,2018,41


In [46]:

for ind, row in df.iterrows():
    # Get index and shareholding
    stock_code_ind = row['stock_code_ind']
    shareholder_ind = row['shareholder_ind']
    shareholding = row['shareholding']
    
    # Assign to sparse matrix
    sparse_matrix[stock_code_ind, shareholder_ind] += shareholding
    

8730059064570.0